In [1]:
%cd ..

/Users/danieloliveiradebrito/Projetos/cyanobacteria-monitoring


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import yaml
import pandas as pd

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn.model_selection import KFold

import plotly.express as px

import numpy as np

from src.data import preprocess

import optuna
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_optimization_history

import s3fs

In [4]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

xgb_p = config["train"]["estimators"]["xgboost"]["params"]

fs = s3fs.S3FileSystem()

In [5]:
for k, v in xgb_p.items():
    print(k, v)

n_estimators 1000
learning_rate 0.5
max_depth 16
subsample 0.8
colsample_bytree 0.9
colsample_bylevel 0.4
scale_pos_weight 85


# Load Data

In [6]:
df = pd.read_parquet(fs.open(config["featurize"]["ft_data_path"]))

In [7]:
df_train, df_test = preprocess.train_test_split_ts(df, config)

22.52% dos dados utilizados para teste


In [8]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=3)
for train_index, val_index in tscv.split(df_train):
        df_train_2, df_val = df_train.iloc[train_index], df_train.iloc[val_index]

In [11]:
from src.data import preprocess

features = config["featurize"]["selected_features"]

X_train = df_train[features]
X_test = df_test[features]
y_train = df_train[config["featurize"]["target_column"]]
y_test = df_test[config["featurize"]["target_column"]]


# Train

## Ridge

In [25]:
# Poly Ridge
RANDOM = config["base"]["random_state"]

def objective(trial, X, y):

    param_grid = {
        "alpha": trial.suggest_float("alpha", .5, 100),
        "degree": trial.suggest_int("degree", 2, 5)
    }

    poly = PolynomialFeatures(degree=param_grid["degree"], include_bias=False)
    ridge_reg = Ridge(param_grid["alpha"])

    poly_ridge = Pipeline([
        ("poly", poly),
        ("regressor", ridge_reg)
    ])

    model = TransformedTargetRegressor(regressor = poly_ridge)

    cv = RepeatedKFold(n_repeats=2, n_splits=3, random_state=RANDOM)

    cv_scores = []
    for (train_index, val_index) in cv.split(X, y):
    #print(train_index, val_index)
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_train, y_train)
                
        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study = optuna.create_study(direction="minimize", study_name="Ridge")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials=50)

[I 2024-01-06 12:19:55,208] A new study created in memory with name: Ridge
[I 2024-01-06 12:19:55,241] Trial 0 finished with value: 3444.8965674487063 and parameters: {'alpha': 94.32737153398172, 'degree': 2}. Best is trial 0 with value: 3444.8965674487063.


[I 2024-01-06 12:19:55,317] Trial 1 finished with value: 10530.601014736014 and parameters: {'alpha': 65.25797515889998, 'degree': 4}. Best is trial 0 with value: 3444.8965674487063.
[I 2024-01-06 12:19:55,346] Trial 2 finished with value: 3436.38525178056 and parameters: {'alpha': 75.14070322519837, 'degree': 2}. Best is trial 2 with value: 3436.38525178056.
[I 2024-01-06 12:19:55,381] Trial 3 finished with value: 3475.936796889435 and parameters: {'alpha': 16.1087641748673, 'degree': 2}. Best is trial 2 with value: 3436.38525178056.
[I 2024-01-06 12:19:55,427] Trial 4 finished with value: 3430.509995964729 and parameters: {'alpha': 51.49420785702814, 'degree': 2}. Best is trial 4 with value: 3430.509995964729.
[I 2024-01-06 12:19:55,596] Trial 5 finished with value: 41244.128050966785 and parameters: {'alpha': 78.12743529030092, 'degree': 5}. Best is trial 4 with value: 3430.509995964729.
[I 2024-01-06 12:19:55,679] Trial 6 finished with value: 12970.552852472749 and parameters: {'al

In [26]:
plot_param_importances(study)

In [27]:
study.best_params

{'alpha': 50.19069909493011, 'degree': 2}

In [28]:
poly = PolynomialFeatures(degree=study.best_params["degree"], include_bias=False)
ridge_reg = Ridge(alpha=study.best_params["alpha"])

poly_ridge = Pipeline([
        ("poly", poly),
        ("regressor", ridge_reg)
    ])

model = TransformedTargetRegressor(regressor = poly_ridge)

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([-2169.25786867, -5110.53046397, -4504.2498761 ])

In [29]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

px.scatter(
    pd.DataFrame(dict(
        y_true=y_test,
        y_pred=y_pred
    )),
    y="y_true",
    x="y_pred"
)

In [32]:
RANDOM = 42

def objective_xgb(trial, X, y):

    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1),
        "min_child_weight": trial.suggest_int("min_child_weigth", 1, 10),
        "max_depth": trial.suggest_int("max_depth", 1, 50),
        "subsample": trial.suggest_float("subsample", .1, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", .1, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", .1, 1),
        "num_leaves": trial.suggest_int("num_leaves", 2**12, 2**16, step=20), 
        "reg_lambda": trial.suggest_int("lambda_l2", 1e-9, 100),
        "reg_alpha": trial.suggest_int("lambda_l1", 1e-9, 100),
        "gamma": trial.suggest_float("min_gain_to_split", 1e-9, .5),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-6, 500),
        "random_state": RANDOM
        }

    cv = KFold(n_splits=3, random_state=RANDOM, shuffle=True)
    cv_scores = []

    xgb = xgboost.XGBRegressor(**param_grid, objective="count:poisson")

    poly = PolynomialFeatures(degree=2, include_bias=False)

    poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", xgb)
    ])

    
    for (train_index, val_index) in cv.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]    

        poly_reg.fit(
            X_train,
            y_train
        )

        preds = poly_reg.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study_xgb = optuna.create_study(direction="minimize", study_name="XGBoost Regression")
func_xgb = lambda trial: objective_xgb(trial, X_train, y_train)
study_xgb.optimize(func_xgb, n_trials=50)

[I 2024-01-06 12:26:14,609] A new study created in memory with name: XGBoost Regression


[12:26:14] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:15] WARNING: /Users/runner/work/xgboost/x

[I 2024-01-06 12:26:16,031] Trial 0 finished with value: 46552.383831891326 and parameters: {'n_estimators': 1000, 'learning_rate': 0.6422135996948172, 'min_child_weigth': 5, 'max_depth': 8, 'subsample': 0.10980117348722085, 'colsample_bytree': 0.6197809084531036, 'colsample_bylevel': 0.8373464210864315, 'num_leaves': 12456, 'lambda_l2': 73, 'lambda_l1': 25, 'min_gain_to_split': 0.1791944920063653, 'scale_pos_weight': 277.69924059583224}. Best is trial 0 with value: 46552.383831891326.


[12:26:16] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:16] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:26:17] WARNING: /Users/runner/work/xgboost/x

[I 2024-01-06 12:26:17,683] Trial 1 finished with value: 3336.4196327371255 and parameters: {'n_estimators': 1000, 'learning_rate': 0.5293477219522555, 'min_child_weigth': 1, 'max_depth': 25, 'subsample': 0.4288718937870074, 'colsample_bytree': 0.5579008526079479, 'colsample_bylevel': 0.7822425160858024, 'num_leaves': 50916, 'lambda_l2': 48, 'lambda_l1': 71, 'min_gain_to_split': 0.08505692226146298, 'scale_pos_weight': 266.5574508464907}. Best is trial 1 with value: 3336.4196327371255.


...

[I 2024-01-06 12:27:55,099] Trial 47 finished with value: 3254.9262791190827 and parameters: {'n_estimators': 1000, 'learning_rate': 0.010369727783246295, 'min_child_weigth': 5, 'max_depth': 19, 'subsample': 0.8565325583957363, 'colsample_bytree': 0.9215047409730066, 'colsample_bylevel': 0.1666976171614789, 'num_leaves': 39916, 'lambda_l2': 27, 'lambda_l1': 92, 'min_gain_to_split': 0.4855724775114277, 'scale_pos_weight': 406.4992996851986}. Best is trial 31 with value: 3151.9427515098027.


[12:27:55] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:56] WARNING: /Users/runner/work/xgboost/x

[I 2024-01-06 12:27:57,716] Trial 48 finished with value: 3412.978374012879 and parameters: {'n_estimators': 1000, 'learning_rate': 0.11630973076590659, 'min_child_weigth': 5, 'max_depth': 19, 'subsample': 0.9408336456321826, 'colsample_bytree': 0.949238450274242, 'colsample_bylevel': 0.2805057561617365, 'num_leaves': 44756, 'lambda_l2': 25, 'lambda_l1': 31, 'min_gain_to_split': 0.44296608855449116, 'scale_pos_weight': 473.88467799205665}. Best is trial 31 with value: 3151.9427515098027.


[12:27:57] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:27:59] WARNING: /Users/runner/work/xgboost/x

[I 2024-01-06 12:28:00,015] Trial 49 finished with value: 3329.561848487173 and parameters: {'n_estimators': 1000, 'learning_rate': 0.07161606766420271, 'min_child_weigth': 6, 'max_depth': 13, 'subsample': 0.849396621327758, 'colsample_bytree': 0.5856997893516803, 'colsample_bylevel': 0.35215205041831116, 'num_leaves': 43616, 'lambda_l2': 13, 'lambda_l1': 88, 'min_gain_to_split': 0.49893111881863006, 'scale_pos_weight': 239.70359235339043}. Best is trial 31 with value: 3151.9427515098027.


In [33]:
plot_optimization_history(study_xgb)

In [34]:
plot_param_importances(study_xgb)

In [35]:
study_xgb.best_params

{'n_estimators': 1000,
 'learning_rate': 0.01325764689971859,
 'min_child_weigth': 10,
 'max_depth': 32,
 'subsample': 0.6041796412576702,
 'colsample_bytree': 0.8445812301185116,
 'colsample_bylevel': 0.6893451676758633,
 'num_leaves': 25576,
 'lambda_l2': 42,
 'lambda_l1': 67,
 'min_gain_to_split': 0.009054274582836606,
 'scale_pos_weight': 483.0526300092115}

In [36]:
poly = PolynomialFeatures(degree=2, include_bias=False)

reg = xgboost.XGBRegressor(**study_xgb.best_params, objective="count:poisson")

poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", reg)
    ])

cross_val_score(poly_reg, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

[12:29:18] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:29:19] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
 

array([ -699.81444419, -3123.59235589, -5433.21269491])

In [38]:
poly_reg.fit(X_train, y_train)
y_pred = poly_reg.predict(X_test)

px.scatter(
    pd.DataFrame(dict(
        y_true=y_test,
        y_pred=y_pred
    )),
    y="y_true",
    x="y_pred"
)

[12:31:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [39]:
def objective_rf(trial, X, y):

    param_grid = {
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000)
        }

    cv = KFold(n_splits=3, random_state=42, shuffle=True)
    cv_scores = []

    rf = RandomForestRegressor(**param_grid, criterion="absolute_error")

    poly = PolynomialFeatures(degree=3, include_bias=False)

    poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", rf)
    ])

    model = TransformedTargetRegressor(
        regressor = poly_reg,
        transformer=PowerTransformer(method="yeo-johnson")
        )
    
    for (train_index, val_index) in cv.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]    

        model.fit(
            X_train,
            y_train
        )

        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study_rf = optuna.create_study(direction="minimize", study_name="Random Forest Regression")
func_rf = lambda trial: objective_rf(trial, X_train, y_train)
study_rf.optimize(func_rf, n_trials=50)

[I 2024-01-06 12:31:47,495] A new study created in memory with name: Random Forest Regression


[I 2024-01-06 12:31:48,047] Trial 0 finished with value: 3188.8955997220014 and parameters: {'max_features': 'log2', 'min_samples_leaf': 10, 'n_estimators': 311}. Best is trial 0 with value: 3188.8955997220014.
[I 2024-01-06 12:32:26,842] Trial 1 finished with value: 3150.7190487945113 and parameters: {'max_features': None, 'min_samples_leaf': 5, 'n_estimators': 684}. Best is trial 1 with value: 3150.7190487945113.
[I 2024-01-06 12:32:30,667] Trial 2 finished with value: 3182.768436520332 and parameters: {'max_features': 'sqrt', 'min_samples_leaf': 7, 'n_estimators': 1311}. Best is trial 1 with value: 3150.7190487945113.
[I 2024-01-06 12:32:32,113] Trial 3 finished with value: 3183.0913403740874 and parameters: {'max_features': 'sqrt', 'min_samples_leaf': 7, 'n_estimators': 506}. Best is trial 1 with value: 3150.7190487945113.
[I 2024-01-06 12:34:16,934] Trial 4 finished with value: 3154.098174894318 and parameters: {'max_features': None, 'min_samples_leaf': 2, 'n_estimators': 1296}. B

In [40]:
study_rf.best_params

{'max_features': 'log2', 'min_samples_leaf': 4, 'n_estimators': 1746}

In [41]:
poly = PolynomialFeatures(degree=3, include_bias=False)

rf_best_params = study_rf.best_params


reg = RandomForestRegressor(**rf_best_params, criterion="absolute_error")

poly_reg = Pipeline([
        ("poly", poly),
        ("regressor", reg)
    ])

model = TransformedTargetRegressor(
    regressor = poly_reg,
    transformer=PowerTransformer(method="yeo-johnson")
)

cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

array([ -955.63160102, -2900.45524273, -5383.49802035])

In [ ]:
study

In [42]:
with open("params.yaml") as config_file:
        config = yaml.safe_load(config_file)

poly = PolynomialFeatures(2, include_bias=False)

xgb_params = study_xgb.best_params
rf_params = study_rf.best_params
ridge_params = {"alpha": 50}

xgb_reg = xgboost.XGBRegressor(**xgb_params)
rf_reg = RandomForestRegressor(**rf_params)
ridge_reg = Ridge(**ridge_params)

def create_model(reg):
    poly_reg = Pipeline([
            ("poly", poly),
            ("regressor", reg)
        ])

    model = TransformedTargetRegressor(
        regressor = poly_reg,
        transformer=PowerTransformer(method="yeo-johnson")
    )

    return model

xgb_model = create_model(xgb_reg)
rf_model = create_model(rf_reg)
ridge_reg = create_model(ridge_reg)

In [43]:
xgb_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
ridge_reg.fit(X_train, y_train)

[12:54:37] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




TransformedTargetRegressor(regressor=Pipeline(steps=[('poly',
                                                      PolynomialFeatures(include_bias=False)),
                                                     ('regressor',
                                                      Ridge(alpha=50))]),
                           transformer=PowerTransformer())

In [44]:
y_pred_xgb = xgb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_ridge = ridge_reg.predict(X_test)

In [45]:
mean_absolute_error(y_test, y_pred_xgb)

2331.4862870407105

In [46]:
y_pred = y_pred_ridge

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

In [47]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso

estimators = [
    ('ridge', ridge_reg),
    ('xgboost', xgb_reg),
    #('lasso', Lasso())
    #('knr', KNeighborsRegressor(n_neighbors=4, metric='minkowski'))
    ]

stack_reg = StackingRegressor(
    estimators = estimators,
    final_estimator=RidgeCV()
)

stack_model = create_model(stack_reg)

cross_val_score(stack_model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

[12:55:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:55:51] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Pleas

array([-1939.60670665, -2768.78202828, -5475.82208912])

In [48]:
stack_model.fit(X_train, y_train)
y_stack = stack_model.predict(X_test)
mean_absolute_error(y_test, y_stack)

[12:56:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:56:17] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Pleas

2644.9508489010577

In [49]:
y_pred = y_stack

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

In [50]:
from sklearn.ensemble import VotingRegressor

vr = VotingRegressor(estimators=estimators)

voting_model = create_model(vr)

cross_val_score(voting_model, X_train, y_train, cv=3, scoring="neg_mean_absolute_error")

[12:56:46] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[12:56:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Pleas

array([-1088604.03271592,   -10788.65317568,    -5711.83958714])

In [51]:
voting_model.fit(X_train, y_train)
y_voting = stack_model.predict(X_test)
print(mean_absolute_error(y_test, y_voting))

y_pred = y_voting

fig = px.scatter(x = y_pred, y = y_test)
fig.update_layout(shapes=[{"type": "line", "y0":0, "y1":max(y_pred), "x0":0, "x1": max(y_pred)}])

[12:56:52] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "lambda_l1", "lambda_l2", "min_child_weigth", "min_gain_to_split", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


2644.9508489010577


In [116]:
with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

pred = pd.read_csv(config["evaluate"]["final_predictions_file"])

pred.head()

,date,y_pred
0,2018-12-24 13:40:42,316.016800
1,2018-12-26 13:30:43,47.188125
2,2018-12-29 13:40:40,311.473424
3,2019-01-25 13:30:47,72.442495
4,2019-01-28 13:40:43,220.278946


In [123]:
from src.data.label_gee import load_data

_, ciano = load_data("params.yaml")

2022-08-27 13:50:41,454 — DATA_LABEL — INFO — Reading data
2022-08-27 13:50:41,505 — DATA_LABEL — INFO — Filtering data


c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [126]:
ciano.head()

,Data da coleta,Resultado
4873,2018-12-19,719
2949,2019-01-18,0
2538,2019-01-28,251
4881,2019-02-11,335
3723,2019-02-25,298


In [52]:
ridge_reg.fit(X_train, y_train)

TransformedTargetRegressor(regressor=Pipeline(steps=[('poly',
                                                      PolynomialFeatures(include_bias=False)),
                                                     ('regressor',
                                                      Ridge(alpha=50))]),
                           transformer=PowerTransformer())

In [154]:
#https://stackoverflow.com/questions/62287001/how-to-overlay-two-plots-in-same-figure-in-plotly-create-pareto-chart-in-plotl


import plotly.graph_objects as go
from plotly.subplots import make_subplots

gee_plot = go.Scatter(
    x=pred["date"],
    y=pred["y_pred"],
    name="Predicted values")

vigi_plot = go.Scatter(
    x=ciano["Data da coleta"],
    y=ciano["Resultado"],
    mode="markers",
    name = "SISAGUA")

fig = make_subplots()
fig.add_trace(gee_plot)
fig.add_trace(vigi_plot)

#fig.add_trace(px.scatter(ciano, x="Data da coleta", y = "Resultado"))

In [156]:
from src.report.plot_predicted_values import plot_predicted_values

plot_predicted_values("params.yaml")

2022-08-27 18:46:35,583 — DATA_LABEL — INFO — Reading data
2022-08-27 18:46:35,643 — DATA_LABEL — INFO — Filtering data


c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Daniel\cyanobacteria-monitoring\src\data\label_gee.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
import yaml

with open("params.yaml") as config_file:
    config = yaml.safe_load(config_file)

In [9]:
x = config["featurize"]["selected_clean_columns"]

x + ['Resultado', 'Data da coleta', 'interval']

['date',
 'NDVI_median',
 'NDCI_median',
 'B1_median',
 'B2_median',
 'B3_median',
 'B4_median',
 'B5_median',
 'B6_median',
 'B7_median',
 'B8_median',
 'B8A_median',
 'B9_median',
 'B11_median',
 'B12_median',
 ['Resultado', 'Data da coleta', 'interval'],
 ['Resultado', 'Data da coleta', 'interval'],
 ['Resultado', 'Data da coleta', 'interval'],
 'Resultado',
 'Data da coleta',
 'interval']

In [14]:
selected_columns = config["featurize"]["selected_clean_columns"] 
selected_columns

['date',
 'NDVI_median',
 'NDCI_median',
 'B1_median',
 'B2_median',
 'B3_median',
 'B4_median',
 'B5_median',
 'B6_median',
 'B7_median',
 'B8_median',
 'B8A_median',
 'B9_median',
 'B11_median',
 'B12_median',
 'Resultado',
 'Data da coleta',
 'interval',
 'Resultado',
 'Data da coleta',
 'interval']

In [70]:
from sklearn.linear_model import PoissonRegressor

1970.1362617961097

In [74]:
# Poly Ridge
RANDOM = config["base"]["random_state"]



def objective(trial, X, y):

    param_grid = {
        "alpha": trial.suggest_float("alpha", 0, 1),
    }

    model = PoissonRegressor(max_iter=10_000, alpha=param_grid["alpha"])
    
    cv = RepeatedKFold(n_repeats=2, n_splits=3, random_state=RANDOM)

    cv_scores = []
    for (train_index, val_index) in cv.split(X, y):
    #print(train_index, val_index)
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        model.fit(X_train, y_train)
                
        preds = model.predict(X_val)
        cv_scores.append(mean_absolute_error(preds, y_val))
    
    return np.median(cv_scores)

study = optuna.create_study(direction="minimize", study_name="Poisson")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials=50)

[I 2024-01-06 16:46:28,096] A new study created in memory with name: Poisson


[I 2024-01-06 16:46:28,817] Trial 0 finished with value: 3217.009093788988 and parameters: {'alpha': 0.30065974100680615}. Best is trial 0 with value: 3217.009093788988.
[I 2024-01-06 16:46:29,543] Trial 1 finished with value: 3238.1971786681975 and parameters: {'alpha': 0.39259029829124414}. Best is trial 0 with value: 3217.009093788988.
[I 2024-01-06 16:46:29,979] Trial 2 finished with value: 3329.333484034722 and parameters: {'alpha': 0.9950221112178053}. Best is trial 0 with value: 3217.009093788988.
[I 2024-01-06 16:46:30,620] Trial 3 finished with value: 3278.4675846100426 and parameters: {'alpha': 0.6135661585406824}. Best is trial 0 with value: 3217.009093788988.
[I 2024-01-06 16:46:31,205] Trial 4 finished with value: 3292.837672744864 and parameters: {'alpha': 0.7082706651515595}. Best is trial 0 with value: 3217.009093788988.
[I 2024-01-06 16:46:31,734] Trial 5 finished with value: 3314.461608629376 and parameters: {'alpha': 0.869572687762948}. Best is trial 0 with value: 32

In [75]:
study.best_params

{'alpha': 0.0472607587356381}

In [78]:
pois_reg = PoissonRegressor(max_iter=10_000, alpha=.5)
pois_reg.fit(X_train, y_train)
print(mean_absolute_error(y_test, pois_reg.predict(X_test)))

px.scatter(
    pd.DataFrame(
        dict(
            y_pred=pois_reg.predict(X_test),
            y_true=y_test
        )
    ),
    x="y_pred",
    y="y_true"
)

1970.1362617961097


In [82]:
with open("params.yaml") as f:
    config = yaml.safe_load(f)

In [83]:
today_ = "20230101"
config["evaluate"]["final_predictions_file"].format(dt=today_)

'f"s3://cyano-guaiba/data/prediction_20230101.gzip"'

In [89]:
with open("params.yaml") as f:
    config = yaml.safe_load(f)

In [98]:
np.sort(['cyano-guaiba/data/prediction_20240109.gzip', 'cyano-guaiba/data/prediction_20240115.gzip'])

array(['cyano-guaiba/data/prediction_20240109.gzip',
       'cyano-guaiba/data/prediction_20240115.gzip'], dtype='<U42')

In [108]:
import json
mae_file = open(config["evaluate"]["metrics_file"])
mae = json.load(mae_file)

In [109]:
mae

{'mae': 2570.087376356467, 'mlse': 5.997943771372978}